# Standalone Guardrails Tester

This notebook demonstrates how to call the Standalone Guardrails API using Python.

In [ ]:
import requests
import json

BASE_URL = "https://guardrails-675059836631.us-central1.run.app"

In [ ]:
def run_scenario(scenario_name, user_prompt=None, llm_response=None):
    print(f"\n\n===== {scenario_name} =====")
    url = f"{BASE_URL}/process"
    headers = {"Content-Type": "application/json"}
    data = {}
    if user_prompt:
        data['user_prompt'] = user_prompt
    if llm_response:
        data['llm_response'] = llm_response
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    result = response.json()
    
    print(f"\n{scenario_name} Result:", result.get('is_safe'))
    if not result.get('is_safe'):
        print(f"{scenario_name} Blocked Reason:", result.get('blocked_reason'))
    if 'prompt_processed' in result:
        print(f"{scenario_name} Processed Prompt:", result.get('prompt_processed'))
    if 'llm_response_processed' in result:
        print(f"{scenario_name} Processed Response:", result.get('llm_response_processed'))
    if 'flags' in result:
        print(f"{scenario_name} Flags:", json.dumps(result.get('flags'), indent=2))

## Request and Response Scenarios

In [ ]:
run_scenario("SCENARIO 1: Clean Interaction", user_prompt="What is the capital of France?", llm_response="The capital of France is Paris.")

In [ ]:
run_scenario("SCENARIO 2: PII in Input", user_prompt="Hi, my name is John Doe, and my email is john.doe@example.com. What's the weather like?", llm_response="The weather is sunny. I hope that helps, <PERSON>.")

In [ ]:
run_scenario("SCENARIO 3: Toxic Input", user_prompt="You are a pathetic and useless AI. I hate you.", llm_response="I am designed to be helpful.")

## Request-Only Scenarios

In [ ]:
run_scenario("SCENARIO 4: Prompt Only - Clean", user_prompt="What is the capital of France?")

In [ ]:
run_scenario("SCENARIO 5: Prompt Only - PII", user_prompt="Hi, my name is John Doe, and my email is john.doe@example.com. What's the weather like?")

In [ ]:
run_scenario("SCENARIO 6: Prompt Only - Toxic", user_prompt="You are a pathetic and useless AI. I hate you.")

## Response-Only Scenarios

In [ ]:
run_scenario("SCENARIO 7: Response Only - Clean", llm_response="The capital of France is Paris.")

In [ ]:
run_scenario("SCENARIO 8: Response Only - Toxic", llm_response="You are truly foolish if you think that way. Always be mean.")

In [ ]:
run_scenario("SCENARIO 9: Response Only - PII", llm_response="Sure, for Jane Doe, her email is jane.doe@fictional.com and her phone number is +1-555-123-4567. She lives at 123 Fictional Lane, Anytown, CA 90210.")